In [9]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV

In [2]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\FER2013-augmented')
LABELS = os.listdir(PATH)

In [3]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/face-emotion-recognition"

Repository IdjiotSandwiches/face-emotion-recognition initialized!

In [4]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/128)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_64F
IMAGE_SIZE = (224,224)
N_COMPONENTS = 10
BLUR = (5,5)

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [5]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\train\\down-sampled-augmented\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\train\\down-sampled-augmented\\labels.npy')

In [6]:
images = StandardScaler().fit_transform(images)

In [10]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)
X_train, _, y_train, _ = train_test_split(
    X_train, y_train, train_size=2000, stratify=y_train, random_state=42
)

In [12]:
param_grid = {
    'C': range(1, 5),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [3, 4],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(
    estimator=SVC(probability=True),
    param_grid=param_grid,
    scoring='accuracy',
    n_jobs=4,
    cv=3,
    verbose=2
)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


KeyboardInterrupt: 

In [10]:
model_params = {
    'C': 1.0,
    'penalty': 'l2',
    'dual': False,
    'verbose': True,
    'max_iter': 5000
}

model = LinearSVC(**model_params).fit(X_train, y_train)
model = CalibratedClassifierCV(model, method='sigmoid')
model.fit(X_train, y_train)

predict = model.predict(X_test)
predict_proba = model.predict_proba(X_test)

metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
    'auc_score': roc_auc_score(y_test, predict_proba, multi_class='ovr', average='macro')
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}\nAUC Score: {metrics['auc_score']}")
report = classification_report(y_test, predict)
print(report)

mlflow.set_experiment("SVM_Model FER2013 augmented")
mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

with mlflow.start_run(run_name=f'SVM LinearSVC'):
    mlflow.log_params(gabor_params)
    mlflow.log_params(model_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('PCA_n_components', N_COMPONENTS)
    mlflow.log_param('gaussian_blur', BLUR)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path='SVM Model',
        input_example=X_train[:1]
    )

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Accuracy: 0.15941439609597396
Precision: 0.15072053927217524
Recall: 0.16022863923744973
F1 Score: 0.14074729897377766
AUC Score: 0.5186884924605099
              precision    recall  f1-score   support

       angry       0.14      0.10      0.12       370
     disgust       0.13      0.08      0.10       357
        fear       0.18      0.40      0.25       334
       happy       0.14      0.02      0.04       333
     neutral       0.16      0.21      0.18       365
         sad       0.16      0.22      0.18       353
    surprise       0.15      0.09      0.11       347

    accuracy                           0.16      2459
   macro avg       0.15      0.16      0.14      2459
weighted avg       0.15      0.16      0.14      2459



🏃 View run SVM LinearSVC at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/19/runs/6ee7fa1ffd1b4779a0a1d41b14caf7ad
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/19
